<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# 模块 2.1: 您的第一个 Chisel 模块
**上一节: [Scala 简介](1_intro_to_scala_zh.ipynb)**<br>
**下一节: [组合逻辑](2.2_comb_logic_zh.ipynb)**

## 动机

现在你已经熟悉了 Scala，让我们开始刻画一些硬件吧！Chisel 是 Scala嵌入式语言中构造硬件的缩写。Chisel是 **C**onstructing **H**ardware **I**n a **S**cala **E**mbedded **L**anguage (Scala 嵌入式语言构造硬件) 的缩写。这意味着它是Scala 中的 DSL，允许你在同一代码中同时利用 Scala 和 Chisel 编程的优势。了解哪些代码是 "Scala"，哪些代码是 "Chisel" 是很重要的，但我们将在之后进行更多的讨论。现在，把 Chisel 和模块二中的代码看作是一种更好的编写 Verilog 的方法。这个模块向你抛出了一整个 Chisel 模块和测试器。暂时只需掌握它的要领。你以后会看到更多的例子。

## 设置

下面的单元格下载了Chisel所需的依赖关系。你会在以后的所有笔记本中看到它。**现在运行这个单元格**。

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

正如上一个模块中提到的，导入Chisel需要这些语句。在运行任何未来的代码块之前，**先运行这个单元**。

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

---
# 您的第一个模块

本节将介绍你的第一个硬件模块，一个测试用例，以及如何运行它。它将包含许多你不理解的东西，这没关系。我们希望你能从这里学到一些粗浅的东西，这样你就可以不断地返回到这个完整的、可行的例子中来强化你所学到的东西。

<span style="color:blue">**示例: 一个模块**</span><br>

和 Verilog 一样，我们可以在 Chisel 中声明模块定义。下面的例子是一个 Chisel 模块 `Module`，`Passthrough`，它有一个 4 位输入，`in`，和一个 4 位输出，`out`。该模块组合连接 `in` 和 `out`，所以 `in` 驱动 `out`。

In [ ]:
// Chisel 代码: 声明一个新的模块定义
class Passthrough extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

这里有很多内容！下面从硬件描述的角度来一行行解释：

```scala
class Passthrough extends Module {
```

我们声明了一个叫 `Passthrough` 的模块。 `Module` 是一个内置的 Chisel 类，所有硬件模块都必须扩展。

```scala 
val io = IO(...)
```

我们将所有的输入和输出端口声明在一个特殊的 `io` `val`中。它必须被称为 `io`，并且是一个 `IO` 对象或实例，这就需要类似 `IO(_instantiated_bundle_)` 这样的形式。

```scala
new Bundle {
    val in = Input(...)
    val out = Output(...)
}
```

我们声明一个新的硬件结构类型 (Bundle)，它包含一些命名的信号 `in` 和 `out`，分别有输入和输出方向。

```scala
UInt(4.W)
```

我们声明一个信号的硬件类型。在本例中，它是一个宽度为4的无符号整数。

```scala
io.out := io.in
```

我们将输入端口连接到输出端口，这样 `io.in` 就会 *驱动* `io.out`。请注意，`:=` 运算符是一个 ***Chisel*** 运算符，表示右手边的信号驱动左手边的信号。它是一个有方向性的操作符。

硬件构造语言（HCL）的整洁之处在于，我们可以把底层的编程语言当作脚本语言来使用。例如，在声明了我们的 Chisel 模块后，我们再使用 Scala 调用 Chisel 编译器将 Chisel `Passthrough` 翻译成 Verilog `Passthrough`。这个过程叫做 ***elaboration***。

In [ ]:
// Scala 代码: 将我们的 Chisel 设计翻译为 Verilog
// 不要担心理解这段代码，它是非常复杂的 Scala 代码
println(getVerilog(new Passthrough))

请注意，我们的模块名称是 `cmd<#>WrapperHelperPassthrough`，这是在 Jupyter 中运行本教程的产物。在你正常的代码中，它的名字应该只是 `Passthrough`。这是一个很重要的教训--虽然 Chisel 会尽力保留模块和其他硬件组件的名称，但有时也会失败。

> 译者注：似乎由于底层运行时的升级，现在得到的 Verilog 代码中模块名称不是 `cmd<#>WrapperHelperPassthrough`。

<span style="color:blue">**示例: 一个模块生成器**</span><br>

如果我们把学到的 Scala 知识应用到这个例子中，我们可以看到 Chisel 模块是作为一个 Scala 类来实现的。就像其他 Scala 类一样，我们可以让 Chisel 模块接受一些构造参数。在这种情况下，我们实现一个新的类 `PassthroughGenerator`，它将接受一个整数 `width` 来决定其输入和输出端口的宽度:

In [ ]:
// Chisel 代码，传递参数决定端口宽度
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}

// 让我们来生成不同宽度的模块
println(getVerilog(new PassthroughGenerator(10)))
println(getVerilog(new PassthroughGenerator(20)))

请注意，生成的 Verilog 根据分配给 `width` 参数的值，对输入/输出使用不同的位宽。让我们来研究一下这是如何工作的。因为 Chisel Modules 是普通的 Scala 类，我们可以利用 Scala 类构造函数的强大功能来对我们的设计进行参数化的阐述。

你可能会注意到，这种参数化是由 *Scala* 启用的，而不是 *Chisel* 启用的；Chisel 没有额外的 API 来进行参数化，但设计师可以简单地利用 Scala 的特性来对他/她的设计进行参数化。

因为 `PassthroughGenerator` 不再描述一个单一的模块，而是描述一个由 `width` 参数化的模块家族，所以我们把这个 `Passthrough` 称为 ***generator (生成器)***。

---
# 测试您的硬件

没有测试器，任何硬件模块或生成器都不完整。Chisel 有内置的测试功能，你将在整个 bootcamp 中探索。下面的例子是一个 Chisel 测试套件（test harness），它将值传递到 `Passthrough` 的输入端口 `in` 的实例，并检查在输出端口 `out` 上是否看到相同的值。

<span style="color:blue">**示例: 一个测试**</span><br>

这里有一些高级 Scala 的内容。然而，除了 `poke` 和 `expect` 命令外，你不需要理解任何东西。你可以把剩下的代码看作是编写这些简单测试的简单模板。

In [ ]:
// Scala 代码: `test` 运行单元测试 
// 测试需要一个用户模块，并有一个代码块用于对被测电路应用测试
test(new Passthrough()) { c =>
    c.io.in.poke(0.U)     // 设置输入为值 0
    c.io.out.expect(0.U)  // 断定正确的输出为 0
    c.io.in.poke(1.U)     // 设置输入为值 1
    c.io.out.expect(1.U)  // 断定正确的输出为 1
    c.io.in.poke(2.U)     // 设置输入为值 2
    c.io.out.expect(2.U)  // 断定正确的输出为 2
}
println("SUCCESS!!") // Scala 代码: 如果执行到这，说明我们的测试通过了！


发生了什么？测试接受一个`Passthrough` 模块，为该模块的输入赋值，并检查其输出。要设置一个输入，我们调用 `poke`。要检查一个输出，我们调用 `expect`。如果我们不想将输出与预期值进行比较（没有断言），我们可以 `peek` 输出来代替。

如果所有 `expect` 语句都为真，那么我们的模板代码将返回 pass。

> 注意，`poke` 和 `expect` 使用的是 Chisel 硬件字面符号。这两个操作都期待正确类型的值。
>
> 如果 `poke` 输入的是一个 `UInt()`，你必须提供一个 `UInt` 字面值 (例如：`c.io.in.poke(10.U)`)，同样，如果输入的是一个 `Bool()`，`poke` 将期望`true.B`或`false.B`。



<span style="color:red">**练习: 编写你自己的测试器**</span><br>

编写并执行两个测试，一个测试 `PassthroughGenerator` 宽度为10，另一个测试 `PassthroughGenerator` 宽度为20。每个至少检查两个值：零和指定宽度支持的最大值。请注意，三个问号在 Scala 中具有特殊的含义。在这些练习中，你可能会经常看到它。运行带有 `???` 的代码会产生 `NotImplementedError`。用你自己的代码替换 `???`。

In [ ]:
// Test with width 10

test(???) { c =>
    ???
}

// Test with width 20

test(???) { c =>
    ???
}

println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>解答</strong> (click to toggle displaying it)</label>
<article>
<pre style="background-color:#f7f7f7">
test(new PassthroughGenerator(10)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.io.in.poke(1023.U)
    c.io.out.expect(1023.U)
}

test(new PassthroughGenerator(20)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.io.in.poke(1048575.U)
    c.io.out.expect(1048575.U)
}

</pre></article></div></section></div>

---
# 观察生成的 Verilog/FIRRTL

如果你在理解生成的硬件方面有困难，并且对阅读结构化的 Verilog 和/或 FIRRTL（Chisel 的 IR，它相当于 Verilog 的一个综合子集）感到很舒服，那么你可以试着看一下生成的 Verilog，看看 Chisel 执行的结果。

下面是一个生成 Verilog（你已经看到了）和 FIRRTL 的例子。

In [ ]:
// 观察 Verilog 代码以调试
println(getVerilog(new Passthrough))

In [ ]:
// 观察 Firrtl 代码以调试
println(getFirrtl(new Passthrough))

---
# 你完成了！

[回到顶部](#top)

## <span style="color:red"> 附录：关于 "printf" 调试的说明</span>

[使用打印语句调试](https://stackoverflow.com/a/189570) 并不总是调试的最好方法，但往往是简单的第一步。当一些事物没有按照预期的方式工作时，打印看看发生了什么。

因为 Chisel 生成器是生成硬件的程序，所以打印生成器和电路状态有一些额外的微妙之处。

重要的是要记住你的打印语句何时执行，以及正在打印什么。

你可能想要打印的三种常见场景有一些重要的区别：
- Chisel 生成器在电路生成过程中进行打印
- 在电路模拟过程中打印电路
- 测试期间打印

`println` 是一个内置的 Scala 函数，可以打印到控制台。由于生成的电路是 FIRRTL 或 Verilog，而不是 Scala，所以在电路仿真过程中 **不能** 使用它来打印。

下面的代码块显示了不同的打印方式。

In [ ]:
class PrintingModule extends Module {
    val io = IO(new Bundle {
        val in = Input(UInt(4.W))
        val out = Output(UInt(4.W))
    })
    io.out := io.in

    printf("Print during simulation: Input is %d\n", io.in)
    // chisel printf has its own string interpolator too
    printf(p"Print during simulation: IO is $io\n")

    println(s"Print during generation: Input is ${io.in}")
}

test(new PrintingModule ) { c =>
    c.io.in.poke(3.U)
    c.clock.step(5) // circuit will print
    
    println(s"Print during testing: Input is ${c.io.in.peek()}")
}